<a href="https://colab.research.google.com/github/sinheechan/Project_Chatbot/blob/main/Travel_destination_recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 여행치 추천 시스템

In [28]:
import numpy as np
import pandas as pd

PREVIOUS_MAX_ROWS = pd.options.display.max_rows

pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
pd.options.display.max_columns = 20

np.random.seed(2023)

np.set_printoptions(precision=4, suppress=True)

import matplotlib.pyplot as plt
import matplotlib

plt.rc('figure', figsize=(8, 4))

In [29]:
!wget -q https://raw.githubusercontent.com/kairess/toy-datasets/master/tn_travel_%E1%84%8B%E1%85%A7%E1%84%92%E1%85%A2%E1%86%BC_A.csv
!wget -q https://raw.githubusercontent.com/kairess/toy-datasets/master/tn_traveller_master_%E1%84%8B%E1%85%A7%E1%84%92%E1%85%A2%E1%86%BC%E1%84%80%E1%85%A2%E1%86%A8%20Master_A.csv
!wget -q https://raw.githubusercontent.com/kairess/toy-datasets/master/tn_visit_area_info_%E1%84%87%E1%85%A1%E1%86%BC%E1%84%86%E1%85%AE%E1%86%AB%E1%84%8C%E1%85%B5%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%87%E1%85%A9_A.csv

In [30]:
# 데이터셋 로드
df_place = pd.read_csv('/content/tn_visit_area_info_방문지정보_A.csv')
df_place.head()

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,POI_NM,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD
0,2210300006,a_a015688,5,프로방스마을,2022-10-30,2022-10-30,경기 파주시 탄현면 새오리로 77,경기 파주시 탄현면 성동리 82-1,126.685239,37.790405,...,NaN,60.0,7,Y,9.0,NaN,4.0,4.0,4.0,NaN
1,2208050001,a_a000164,1,친구/친지집,2022-08-05,2022-08-05,NaN,경기도 안산시 단원구 와동,NaN,NaN,...,NaN,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN,41273.0
2,2210030002,a_a004293,2,병점역 1호선,2022-10-03,2022-10-03,경기 화성시 떡전골로 97,경기 화성시 진안동 824-1,127.033200,37.206753,...,병점역,30.0,9,Y,6.0,NaN,4.0,5.0,5.0,NaN
3,2208200003,a_a000491,12,더현대서울,2022-08-20,2022-08-20,서울 영등포구 여의대로 108,서울 영등포구 여의도동 22,126.928446,37.525872,...,NaN,180.0,4,N,2.0,NaN,5.0,5.0,5.0,NaN
4,2208110007,a_a000172,13,강릉중앙시장,2022-08-11,2022-08-11,강원 강릉시 금성로 21,강원 강릉시 성남동 50,128.898624,37.754025,...,신흥수산,30.0,4,N,1.0,NaN,4.0,4.0,4.0,NaN


In [31]:
df_travel = pd.read_csv('/content/tn_travel_여행_A.csv')
df_travel.head()

,TRAVEL_ID,TRAVEL_NM,TRAVELER_ID,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,MVMN_NM,TRAVEL_PERSONA,TRAVEL_MISSION,TRAVEL_MISSION_CHECK
0,a_a014262,A03,a014262,1;11;2;22;,2022-10-29,2022-10-30,자가용,NaN,1;11;2;22;,2;1;22
1,a_a006728,A03,a006728,21;24;5;22;26;27;,2022-10-10,2022-10-11,자가용,NaN,21;24;5;22;26;27;,6;1;22
2,a_a015745,A02,a015745,1;2;6;,2022-10-31,2022-11-03,대중교통 등,NaN,1;2;6;,13;3;28
3,a_a010544,A03,a010544,21;22;24;26;,2022-11-01,2022-11-02,자가용,NaN,21;22;24;26;,1;22;26
4,a_a012058,A03,a012058,1;4;10;12;21;22;26;,2022-11-05,2022-11-06,대중교통 등,수도권 외 거주 39세 이하 커플,1;4;10;12;21;22;26;,1;21;22


In [32]:
df_traveler = pd.read_csv('/content/tn_traveller_master_여행객 Master_A.csv')
df_traveler.head()

,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,...,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM
0,b015583,41,여,20,6,1.0,1.0,5,3.0,NaN,...,1,7,경기도,충남,2인 여행(가족 외),2022-10-29~2022-10-30,1,NaN,NaN,1
1,a001105,30,남,30,6,1.0,1.0,2,3.0,NaN,...,6,5,대전광역시,경기,3인 이상 여행(가족 외),2022-09-03~2022-09-04,3,NaN,NaN,7
2,a001673,41,여,30,4,1.0,2.0,3,NaN,1.0,...,2,7,경기도,서울,자녀 동반 여행,2022-09-18~2022-09-19,10,7.0,1.0,2
3,a017690,41,여,30,6,1.0,1.0,3,3.0,NaN,...,4,5,경기도,서울,3인 이상 여행(가족 외),2022-11-12~2022-11-12,3,1.0,2.0,3
4,c006666,28,여,30,6,1.0,1.0,3,13.0,NaN,...,5,5,인천광역시,경기,나홀로 여행,2022-10-18~2022-10-18,1,7.0,2.0,0


In [33]:
# 데이터셋 3개 병합
# place + travel
df = pd.merge(df_place, df_travel, on='TRAVEL_ID', how='left')
# df + traveler
df = pd.merge(df, df_traveler, on='TRAVELER_ID', how='left')
df.head()

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM
0,2210300006,a_a015688,5,프로방스마을,2022-10-30,2022-10-30,경기 파주시 탄현면 새오리로 77,경기 파주시 탄현면 성동리 82-1,126.685239,37.790405,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
1,2208050001,a_a000164,1,친구/친지집,2022-08-05,2022-08-05,NaN,경기도 안산시 단원구 와동,NaN,NaN,...,3.0,5.0,경기도,경기,2인 여행(가족 외),2022-08-05~2022-08-06,3.0,2.0,1.0,1.0
2,2210030002,a_a004293,2,병점역 1호선,2022-10-03,2022-10-03,경기 화성시 떡전골로 97,경기 화성시 진안동 824-1,127.033200,37.206753,...,7.0,2.0,경기도,경기,나홀로 여행,2022-10-03~2022-10-03,6.0,2.0,1.0,0.0
3,2208200003,a_a000491,12,더현대서울,2022-08-20,2022-08-20,서울 영등포구 여의대로 108,서울 영등포구 여의도동 22,126.928446,37.525872,...,5.0,7.0,울산광역시,서울,2인 가족 여행,2022-08-19~2022-08-20,1.0,3.0,7.0,1.0
4,2208110007,a_a000172,13,강릉중앙시장,2022-08-11,2022-08-11,강원 강릉시 금성로 21,강원 강릉시 성남동 50,128.898624,37.754025,...,2.0,4.0,광주광역시,서울,나홀로 여행,2022-08-10~2022-08-13,1.0,7.0,2.0,0.0


In [34]:
# TRAVEL_ID가  a_a015688인 것만 추출
df[df['TRAVEL_ID'] == 'a_a015688']

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM
0,2210300006,a_a015688,5,프로방스마을,2022-10-30,2022-10-30,경기 파주시 탄현면 새오리로 77,경기 파주시 탄현면 성동리 82-1,126.685239,37.790405,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
7060,2210300002,a_a015688,7,다온무인텔,2022-10-30,2022-10-31,경기 파주시 탄현면 성동로 7-25,경기 파주시 탄현면 성동리 666-3,126.686597,37.780606,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
8788,2210300005,a_a015688,4,헤이리예술마을,2022-10-30,2022-10-30,NaN,경기 파주시 탄현면 법흥리 1652-239,126.698953,37.788174,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
10467,2210310002,a_a015688,9,집,2022-10-31,2022-11-01,NaN,NaN,NaN,NaN,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
13659,2210310001,a_a015688,8,아이노스 출판단지점,2022-10-31,2022-10-31,경기 파주시 문발로 142,경기 파주시 문발동 531-1,126.686457,37.707170,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
18718,2210300004,a_a015688,3,롯데프리미엄아울렛 파주점,2022-10-30,2022-10-30,경기 파주시 문발로 284,경기 파주시 문발동 644,126.692643,37.718115,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
28239,2210300003,a_a015688,2,지혜의숲,2022-10-30,2022-10-30,경기 파주시 회동길 145,경기 파주시 문발동 524-3,126.687060,37.708354,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30154 entries, 0 to 30153
Data columns (total 67 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   VISIT_AREA_ID              30154 non-null  int64  
 1   TRAVEL_ID                  30154 non-null  object 
 2   VISIT_ORDER                30154 non-null  int64  
 3   VISIT_AREA_NM              30154 non-null  object 
 4   VISIT_START_YMD            30154 non-null  object 
 5   VISIT_END_YMD              30154 non-null  object 
 6   ROAD_NM_ADDR               20381 non-null  object 
 7   LOTNO_ADDR                 21859 non-null  object 
 8   X_COORD                    22821 non-null  float64
 9   Y_COORD                    22821 non-null  float64
 10  ROAD_NM_CD                 24 non-null     float64
 11  LOTNO_CD                   2433 non-null   object 
 12  POI_ID                     13872 non-null  object 
 13  POI_NM                     13872 non-null  obj

칼럼이 너무 많아 전처리

In [36]:
# 전처리 1 => 데이터 DB 구조 이미지 참고

df_fil = df[~df['TRAVEL_MISSION_CHECK'].isnull()].copy()
df_fil.loc[:, 'TRAVEL_MISSION_INT'] = df_fil['TRAVEL_MISSION_CHECK'].str.split(';').str[0].astype(int)
df_fil #23898 rows × 68 columns

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT
0,2210300006,a_a015688,5,프로방스마을,2022-10-30,2022-10-30,경기 파주시 탄현면 새오리로 77,경기 파주시 탄현면 성동리 82-1,126.685239,37.790405,...,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0,5
1,2208050001,a_a000164,1,친구/친지집,2022-08-05,2022-08-05,NaN,경기도 안산시 단원구 와동,NaN,NaN,...,5.0,경기도,경기,2인 여행(가족 외),2022-08-05~2022-08-06,3.0,2.0,1.0,1.0,22
2,2210030002,a_a004293,2,병점역 1호선,2022-10-03,2022-10-03,경기 화성시 떡전골로 97,경기 화성시 진안동 824-1,127.033200,37.206753,...,2.0,경기도,경기,나홀로 여행,2022-10-03~2022-10-03,6.0,2.0,1.0,0.0,28
3,2208200003,a_a000491,12,더현대서울,2022-08-20,2022-08-20,서울 영등포구 여의대로 108,서울 영등포구 여의도동 22,126.928446,37.525872,...,7.0,울산광역시,서울,2인 가족 여행,2022-08-19~2022-08-20,1.0,3.0,7.0,1.0,6
4,2208110007,a_a000172,13,강릉중앙시장,2022-08-11,2022-08-11,강원 강릉시 금성로 21,강원 강릉시 성남동 50,128.898624,37.754025,...,4.0,광주광역시,서울,나홀로 여행,2022-08-10~2022-08-13,1.0,7.0,2.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30149,2210220005,a_a008553,5,집,2022-10-22,2022-10-23,NaN,NaN,NaN,NaN,...,3.0,서울특별시,경기,나홀로 여행,2022-10-22~2022-10-22,2.0,1.0,NaN,0.0,6
30150,2208280003,a_a001505,6,카페아쁠뤼스,2022-08-28,2022-08-28,경기 가평군 청평면 신청평로 107,경기 가평군 청평면 대성리 3-5,127.406939,37.728238,...,4.0,서울특별시,경기,3인 이상 여행(가족 외),2022-08-27~2022-08-28,1.0,3.0,7.0,7.0,5
30151,2210290005,a_a013660,4,국립중앙박물관,2022-10-29,2022-10-29,서울 용산구 서빙고로 137,서울 용산구 용산동6가 168-6,126.980203,37.523847,...,6.0,경기도,서울,나홀로 여행,2022-10-29~2022-10-30,1.0,7.0,8.0,0.0,3
30152,2208060008,a_a000019,7,송도달빛축제공원,2022-08-06,2022-08-06,인천 연수구 센트럴로 350,인천 연수구 송도동 26-1,126.634896,37.407218,...,3.0,서울특별시,인천,3인 이상 여행(가족 외),2022-08-06~2022-08-07,7.0,5.0,1.0,6.0,9


In [37]:
# 전처리 2
# 중요한 용어
'''
성별 나이
트레블스타일1 (자연 vs 도시)
트레블스타일2 (숙박 vs 당일)
트레블스타일3 (새로운지역 vs 익숙)
트레블스타일4 (편하지만 비싼 vs 불편하지만 저비용)
트레블스타일5 (휴양, 체험)
트레블스타일6 (잘 알려지지 않은 vs 유명)
트레블스타일7 (계획 vs 무계획)
트레블스타일8 (사진 중요 vs 사진 안중요)
트레블모티브 (TMT)
동반자 수
트레블미션_INT
방문지 명
만족도
'''
df_fil = df_fil[[
    'GENDER', 'AGE_GRP',
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1',
    'TRAVEL_COMPANIONS_NUM',
    'TRAVEL_MISSION_INT',
    'VISIT_AREA_NM',
    'DGSTFN',
]]

# 필요한 칼럼만 뽑고 dropna
df_fil = df_fil.dropna()
df_fil # 14930 rows × 15 columns

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT,VISIT_AREA_NM,DGSTFN
0,남,30.0,4.0,4.0,2.0,4.0,6.0,5.0,6.0,6.0,1.0,0.0,5,프로방스마을,4.0
2,여,40.0,4.0,3.0,6.0,3.0,2.0,2.0,7.0,2.0,6.0,0.0,28,병점역 1호선,4.0
3,여,30.0,4.0,4.0,4.0,3.0,4.0,4.0,5.0,7.0,1.0,1.0,6,더현대서울,5.0
4,남,30.0,7.0,1.0,1.0,1.0,7.0,7.0,2.0,4.0,1.0,0.0,2,강릉중앙시장,4.0
5,남,20.0,6.0,1.0,1.0,3.0,4.0,6.0,3.0,2.0,3.0,1.0,11,청계천,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30145,여,30.0,3.0,1.0,3.0,3.0,4.0,4.0,7.0,6.0,1.0,0.0,3,행궁동벽화마을,2.0
30150,여,30.0,1.0,3.0,1.0,3.0,4.0,4.0,5.0,4.0,1.0,7.0,5,카페아쁠뤼스,5.0
30151,남,20.0,3.0,6.0,2.0,5.0,4.0,4.0,5.0,6.0,1.0,0.0,3,국립중앙박물관,5.0
30152,남,20.0,5.0,2.0,2.0,4.0,4.0,5.0,5.0,3.0,7.0,6.0,9,송도달빛축제공원,4.0


In [38]:
# 전처리 3
# 수치형 데이터가 아닌 것들을 전처리
cat_features_names = [
    'GENDER',
    #'AGE_GRP',
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1',
    #'TRAVEL_COMPANIONS_NUM',
    'TRAVEL_MISSION_INT',
    'VISIT_AREA_NM',
    #'DGSTFN',
]

# int 형으로 변환
df_fil[cat_features_names[1:-1]] = df_fil[cat_features_names[1:-1]].astype(int)
df_fil

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT,VISIT_AREA_NM,DGSTFN
0,남,30.0,4,4,2,4,6,5,6,6,1,0.0,5,프로방스마을,4.0
2,여,40.0,4,3,6,3,2,2,7,2,6,0.0,28,병점역 1호선,4.0
3,여,30.0,4,4,4,3,4,4,5,7,1,1.0,6,더현대서울,5.0
4,남,30.0,7,1,1,1,7,7,2,4,1,0.0,2,강릉중앙시장,4.0
5,남,20.0,6,1,1,3,4,6,3,2,3,1.0,11,청계천,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30145,여,30.0,3,1,3,3,4,4,7,6,1,0.0,3,행궁동벽화마을,2.0
30150,여,30.0,1,3,1,3,4,4,5,4,1,7.0,5,카페아쁠뤼스,5.0
30151,남,20.0,3,6,2,5,4,4,5,6,1,0.0,3,국립중앙박물관,5.0
30152,남,20.0,5,2,2,4,4,5,5,3,7,6.0,9,송도달빛축제공원,4.0


In [39]:
# 데이터셋 분류
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df_fil, test_size = 0.2, random_state=2023)
print(train_data.shape)
print(test_data.shape)

(11944, 15)
(2986, 15)


In [40]:
!pip install -q catboost # 범주 분류할때 1티어 / 범주형 데이터가 많을 떄 쓰면 일반적으로 좋음

In [41]:
# Catboost
# 타겟은 DDGSTFN : 만족도

from catboost import CatBoostRegressor, Pool
train_pool = Pool(train_data.drop(['DGSTFN'], axis=1),# 열방향
                  label = train_data['DGSTFN'],
                  cat_features = cat_features_names)

test_pool = Pool(test_data.drop(['DGSTFN'], 1),# 열방향
                 label = test_data['DGSTFN'],
                 cat_features = cat_features_names)

<ipython-input-41-a1da5f1d4e41>:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  test_pool = Pool(test_data.drop(['DGSTFN'], 1),# 열방향


In [42]:
# 모델 생성 : 회귀분석

model = CatBoostRegressor(
    loss_function = 'RMSE', # 평균제곱근오차
    eval_metric='MAE', # 평균오차
    task_type='CPU',
    depth=6,
    learning_rate=0.01,
    n_estimators=2000)

# 피팅
model.fit(
    train_pool,
    eval_set = test_pool,
    verbose=500,
    plot=True )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6846896	test: 0.6992830	best: 0.6992830 (0)	total: 88.8ms	remaining: 2m 57s
500:	learn: 0.6099701	test: 0.6130200	best: 0.6130200 (500)	total: 23.8s	remaining: 1m 11s
1000:	learn: 0.5998829	test: 0.6071983	best: 0.6071967 (995)	total: 39.1s	remaining: 39s
1500:	learn: 0.5925082	test: 0.6030121	best: 0.6030121 (1500)	total: 1m 4s	remaining: 21.5s
1999:	learn: 0.5863506	test: 0.6012453	best: 0.6012453 (1999)	total: 1m 21s	remaining: 0us

bestTest = 0.6012453332
bestIteration = 1999



In [43]:
# test셋에서 두번째로 배정된 것 추출
# 실제 체크된 점수와 본래 DB에 나와있는 숫자와 다를 가능성이 있음

test_data.iloc[2]

GENDER                       남
AGE_GRP                   20.0
TRAVEL_STYL_1                2
TRAVEL_STYL_2                2
TRAVEL_STYL_3                3
TRAVEL_STYL_4                2
TRAVEL_STYL_5                3
TRAVEL_STYL_6                3
TRAVEL_STYL_7                6
TRAVEL_STYL_8                5
TRAVEL_MOTIVE_1              3
TRAVEL_COMPANIONS_NUM      1.0
TRAVEL_MISSION_INT           2
VISIT_AREA_NM            한국민속촌
DGSTFN                     5.0
Name: 11830, dtype: object

In [44]:
# 모델 테스트
model.predict(test_data.iloc[2].drop(['DGSTFN']))

4.427927312087277

예측치는 4.4가 나왔지만 실제 만족도는 위에서 추출했듯이 5점이 나왔다.

In [45]:
# 테스트데이터 갯수 확인 (1~5점)

test_data[test_data['DGSTFN'] == 3].index

Int64Index([20533, 27202, 18617,  2449, 26819, 18767, 29695,  6653, 28684,
             3118,
            ...
             8950, 25329, 13180, 21506, 28081,  7903,  3882, 28443, 27245,
            28106],
           dtype='int64', length=380)

1~2점이 별로 없다.

3~5점은 상당 수 분포한다.

In [46]:
# 길이 출력해보기 만족도 점수별로 for 문으로

In [47]:
# 피처 중요도 출력

model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,TRAVEL_MISSION_INT,14.313014
1,VISIT_AREA_NM,10.752435
2,TRAVEL_STYL_6,9.658239
3,TRAVEL_STYL_4,9.415209
4,TRAVEL_STYL_8,9.414176
5,TRAVEL_STYL_7,8.855137
6,TRAVEL_MOTIVE_1,7.655500
7,TRAVEL_STYL_2,7.549269
8,TRAVEL_STYL_1,7.325958
9,TRAVEL_STYL_5,6.990709


In [48]:
# 수도권 데이터 출력
area_names = df_fil[['VISIT_AREA_NM']].drop_duplicates()
area_names

,VISIT_AREA_NM
0,프로방스마을
2,병점역 1호선
3,더현대서울
4,강릉중앙시장
5,청계천
...,...
30128,설악버스터미널
30136,탄도어촌체험마을
30138,하남ICEBOX
30141,부평역사박물관


In [49]:
# 여행자를 샘플로 내가 직접 커스텀 => 문서 보고 코드 정해서 임의로 넣기

traveler = {
    'GENDER':'남',
    'AGE_GRP' : 28.0,
    'TRAVEL_STYL_1' : 1,
    'TRAVEL_STYL_2' : 5,
    'TRAVEL_STYL_3' : 2,
    'TRAVEL_STYL_4' : 2,
    'TRAVEL_STYL_5' : 2,
    'TRAVEL_STYL_6' : 5,
    'TRAVEL_STYL_7' : 1,
    'TRAVEL_STYL_8' : 6,
    'TRAVEL_MOTIVE_1' : 2,
    'TRAVEL_COMPANIONS_NUM' : 4.0,
    'TRAVEL_MISSION_INT' : 5 }

results = pd.DataFrame([], columns=['AREA', 'SCORE'])
for area in area_names['VISIT_AREA_NM']:
  input = list(traveler.values())
  input.append(area)
  score = model.predict(input)
  results = pd.concat([results, pd.DataFrame([[area,score]], columns=['AREA','SCORE'])])
results.sort_values('SCORE', ascending=False)[:10]

,AREA,SCORE
0,수서역,4.500175
0,송도센트럴파크,4.487226
0,화담숲,4.483919
0,스타필드 하남,4.472993
0,창덕궁,4.469330
0,경복궁,4.468234
0,용산역,4.465142
0,국립현대미술관 서울관,4.464545
0,덕수궁,4.458352
0,창경궁,4.457963


In [50]:
results.sort_values('SCORE', ascending=True)[:10]

,AREA,SCORE
0,프로방스마을,4.007158
0,성수역 2호선,4.021561
0,임진각관광지,4.055318
0,LCDC서울,4.058015
0,양평물맑은시장,4.061297
0,남대문시장,4.068410
0,오산EBS캐릭터공원,4.068542
0,울산역,4.073516
0,미리내성지,4.076563
0,아트포레스트,4.077273


In [51]:
# TSY
tsy = pd.read_csv('/content/drive/MyDrive/Dataset/Travel/tc_codeb_코드B.csv')
tsy = tsy[173:179]
tsy

,idx,cd_nm,cd_a,cd_b,cd_memo,cd_memo2,del_flag,order_num,ins_dt,edit_dt
173,1165,자연 선호 매우선호,TSY,1,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
174,1166,자연 선호 중간선호,TSY,2,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
175,1167,자연 선호 약간선호,TSY,3,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
176,1168,중립,TSY,4,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
177,1169,도시 선호 약간선호,TSY,5,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
178,1170,도시 선호 중간선호,TSY,6,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN


In [53]:
# TMT
tmt = pd.read_csv('/content/drive/MyDrive/Dataset/Travel/tc_codeb_코드B.csv')
tmt = tmt[180:190]
tmt

,idx,cd_nm,cd_a,cd_b,cd_memo,cd_memo2,del_flag,order_num,ins_dt,edit_dt
180,1172,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",TMT,1,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
181,1173,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",TMT,2,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
182,1174,여행 동반자와의 친밀감 및 유대감 증진,TMT,3,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
183,1175,진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,TMT,4,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
184,1176,SNS 사진 등록 등 과시,TMT,5,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
185,1177,"운동, 건강 증진 및 충전",TMT,6,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
186,1178,새로운 경험 추구,TMT,7,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
187,1179,"역사 탐방, 문화적 경험 등 교육적 동기",TMT,8,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
188,1180,"특별한 목적(칠순여행, 신혼여행, 수학여행, 인센티브여행)",TMT,9,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
189,1181,기타,TMT,10,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN


In [54]:
# MIS
mis = pd.read_csv('/content/drive/MyDrive/Dataset/Travel/tc_codeb_코드B.csv')
mis = mis[105:126]
mis

,idx,cd_nm,cd_a,cd_b,cd_memo,cd_memo2,del_flag,order_num,ins_dt,edit_dt
105,1097,쇼핑,MIS,1,NaN,NaN,N,999,2022-07-27 16:26:52.000,NaN
106,1098,"테마파크, 놀이시설, 동/식물원 방문",MIS,2,NaN,NaN,N,999,2022-07-27 16:26:58.000,NaN
107,1099,역사 유적지 방문,MIS,3,NaN,NaN,N,999,2022-07-27 16:27:03.000,NaN
108,1100,시티투어,MIS,4,NaN,NaN,N,999,2022-07-27 16:27:08.000,NaN
109,1101,"야외 스포츠, 레포츠 활동",MIS,5,NaN,NaN,N,999,2022-07-27 16:27:14.000,NaN
...,...,...,...,...,...,...,...,...,...,...
121,1113,신규 여행지 발굴,MIS,24,NaN,NaN,N,999,2022-07-27 16:28:21.000,NaN
122,1114,반려동물 동반 여행,MIS,25,NaN,NaN,N,999,2022-07-27 16:28:26.000,NaN
123,1115,인플루언서 따라하기 여행,MIS,26,NaN,NaN,N,999,2022-07-27 16:28:34.000,NaN
124,1116,친환경 여행(플로깅 여행),MIS,27,NaN,NaN,N,999,2022-07-27 16:30:10.000,NaN
